In [0]:
## ALGORITMO GENETICO COMPACTO e SIMPLES BINARIO MATRIZ COVARIANCIA
import random
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy
plt.style.use('ggplot')

mc_sga = {}
mc_cga = {}

###################          SGA        ###################

def generate_pop_sga(size,population_size):
    sga_vector_pop = []

    for ps in range(population_size):
      sga_value = ''
      for s in range(size):
        sga_value += '1' if random.random() > 0.5  else '0'
      sga_vector_pop.append( {'val':sga_value,'fit':0} )

    return sga_vector_pop

def crossover_sga(size,population,mutation):
    son = population[0].copy()
    son2 = population[1].copy()

    s = son['val']
    s2 = son2['val']
    middle = random.randrange(1,size-1)
    
    son['fit'] = 0
    son2['fit'] = 0
    son['val'] = s[:middle] + s2[middle:]
    son2['val'] = s2[:middle] + s[middle:]
    
    # mutation
    child = son['val']
    child2 = son2['val']

    c1 = ""
    for c in child:
      mut = random.random()
      if(mut < mutation):
        c1+='1' if c == '0' else '0'
      else:
        c1+=c
    son['val'] = c1

    c2 = ""
    for s in child2:
      mut = random.random()
      if(mut < mutation):
        c2+='1' if s == '0' else '0'
      else:
        c2+=s
    son2['val'] = c2
    
    return son,son2

def to_Int(individuo):
  l = []
  for i in individuo:
    if i == "1":
      l.append(1)
    else:
      l.append(0)
  return sum(l)

def matriz_covariancia(pop,name):
    n = []
    for i in pop:
      m = []
      for j in i['val']:
        m.append(int(j))
      n.append(m)

    mts = numpy.transpose(n)
    #print(mts)
    mc_sga[name] = numpy.cov(mts)

def run_sga(generations, size, population_size):
    # criação da população aleatoria
    population = generate_pop_sga(size,population_size)
    population = fitness_function_sga(population)

    mutation = 0.01

    matriz_covariancia(population,'inicial')

    # percorre as gerações
    for i in range(generations):

      mid = int(population_size/2) if int(population_size/2) % 2 == 0 else int(population_size/2)-1
      new_population = []
      
      for j in range(0,mid,2):
        new_population += crossover_sga(size,population[j:j+2],mutation)
        #print(j,new_population)

      full_pop = population + new_population
      full_pop = fitness_function_sga(full_pop)

      population = full_pop[:population_size]
      
      if i == int(generations / 2):
        matriz_covariancia(population,'intermediaria')
      
      #print ("generation: {} and population {}".format(i + 1,population))
      p = population[0]
    matriz_covariancia(population,'final')
  
def sort_fit(e):
  return e['fit']

# funcao de fitness
def fitness_function_sga(population):
  vector_fit = []

  for p in population:
    p['fit']= int(p['val'],2) 
    vector_fit.append(p)

  vector_fit.sort(reverse = True,key=sort_fit)
  return vector_fit

###################          CGA        ###################

class Solution(object):
    def __init__(self, value):
        self.value = value
        self.fitness = 0

    def calculate_fitness(self):
        self.fitness = fitness_function(self.value)

def generate_candidate(vector,population_size):
    pop=[]
    for n in range(population_size):
      value = ''
      for q in vector:
        value += "1" if random.random() < q else "0"
      pop.append(value)
    
    #print(pop)
    return Solution(pop)

def generate_vector(size):
    return [0.5] * size

def compete(a, b, a1, b1):
    if a > b:
        return a1, b1
    else:
        return b1, a1

def competition(population):
    for n in range(1,len(population.value)):
      winner,loser = compete(population.fitness[0],population.fitness[n],population.value[0],population.value[n])
    
    return winner, loser

def update_vector(vector, winner, loser, population_size):
    for i in range(len(vector)):
        if winner[i] != loser[i]:
            if winner[i] == '1':
                vector[i] += 1.0 / float(population_size)
            else:
                vector[i] -= 1.0 / float(population_size)

def matriz_covariancia_cga(pop,name):
    n = []
    for i in pop.value:
      m = []
      for j in i:
        m.append(int(j))
      n.append(m)

    mts = numpy.transpose(n)
    mc_cga[name] = numpy.cov(mts)
    
def run_cga(generations, size, population_size):
    # probabilidade do vetor
    vector = generate_vector(size)
    #print(vector)
    best = None
    for i in range(generations):
      population = generate_candidate(vector, population_size)
      population.calculate_fitness()
      winner, loser = competition(population)
      if i == 0:
        matriz_covariancia_cga(population,'inicial')
      
      # verifica qual é o melhor
      if best:
          if int(winner,2) > int(best,2):
             best = winner
      else:
          best = winner
      
      update_vector(vector,winner,loser,population_size)
      
      #print ("generation: {} best value: {} best fitness: {}".format(i + 1, best, int(best,2) ) )
      if i == int(generations / 2):
        matriz_covariancia_cga(population,'intermediaria')

    matriz_covariancia_cga(population,'final')

def fitness_function(value):
  value.sort(reverse=True)

  fit =[]
  for n in value:
    fit.append(int(n,2))

  return fit

def fmt(x, pos):
  return r'{:.2f}'.format(x)

def plotgraphics_sga(name):
  hmmin, hmmax = -0.3, 0.3

  fig, axis = plt.subplots(figsize=(10, 7))
  fig.suptitle('sGA\n Matriz covariancia população '+name,
  fontsize=14, fontweight='bold', y=0.94)
  heatmap = axis.pcolor(mc_sga[name], vmin=hmmin, vmax=hmmax, cmap=plt.cm.Blues)
  plt.colorbar(heatmap, format=ticker.FuncFormatter(fmt))
  plt.show()

def plotgraphics_cga(name):
  hmmin, hmmax = -0.3, 0.3

  fig, axis = plt.subplots(figsize=(10, 7))
  fig.suptitle('cGA\n Matriz covariancia população '+name,
  fontsize=14, fontweight='bold', y=0.94)
  heatmap = axis.pcolor(mc_cga[name], vmin=hmmin, vmax=hmmax, cmap=plt.cm.Reds)
  plt.colorbar(heatmap, format=ticker.FuncFormatter(fmt))
  plt.show()
  

run_sga(10, 10, 10)
run_cga(10, 10, 10)
plotgraphics_sga('inicial')
plotgraphics_sga('intermediaria')
plotgraphics_sga('final')
plotgraphics_cga('inicial')
plotgraphics_cga('intermediaria')
plotgraphics_cga('final')

